# **TP7 - Corresponding Analysis (CA)**

<b>Exploratory Data Analysis & Unsuperivsed Learning </b><br>
**Course: PHAUK Sokkey, PhD** <br> 
**TP: HAS Sothea, PhD**

-------

**Objective**: Qualitative columns are often ignored in predictive models or analysis. It is important to notice that qualitative variables are as important as the quantitative ones when it comes to building predictive models or analyzing their connection within the dataset. In this TP, we will focus on identifying the associations between two qualitative variables.

---------

> **The `Jupyter Notebook` for this TP can be downloaded here: [TP7_CA.ipynb](https://hassothea.github.io/EDA_ITC/TPs/TP7_CA.ipynb)**.

-------

## 1. Data loading and Preprocessing

`Titanic` dataset contains information on the passengers aboard the RMS Titanic, which sank in 1912. It includes details like age, gender, class, and survival status.

**A.** Import the `Titanic` dataset from kaggle using: [Titanic dataset](https://www.kaggle.com/datasets/surendhan/titanic-dataset).

- How many quantitative and qualitative variables are there in this dataset?
- Convert each column into its correct data type.

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("surendhan/titanic-dataset")

In [1]:
# Import data
import pandas as pd
# data = pd.read_csv(path + "/titanic.csv")
data = pd.read_csv("titanic.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [9]:
# Select columns with dtype 'object'
object_columns = data.select_dtypes(include='object').columns.tolist()
print("Quantitative Columns:")
print(object_columns)
print()
print("Qualitative Columns:")
numerical_columns = data.select_dtypes(include='number').columns.tolist()
print(numerical_columns)

Quantitative Columns:
['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

Qualitative Columns:
['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


In [7]:
# Convert columns to appropriate types
df = data.copy()
df['PassengerId'] = df['PassengerId'].astype(str)  # treat as ID
df['Survived'] = df['Survived'].astype('category')  # binary outcome
df['Pclass'] = df['Pclass'].astype('category')  # ordered class
df['Name'] = df['Name'].astype(str)
df['Sex'] = df['Sex'].astype('category')
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')  # ensure numeric, may have NaNs
df['SibSp'] = df['SibSp'].astype(int)
df['Parch'] = df['Parch'].astype(int)
df['Ticket'] = df['Ticket'].astype(str)
df['Fare'] = pd.to_numeric(df['Fare'], errors='coerce')
df['Cabin'] = df['Cabin'].astype(str)  # could also be categorical, often messy
df['Embarked'] = df['Embarked'].astype('category')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  418 non-null    object  
 1   Survived     418 non-null    category
 2   Pclass       418 non-null    category
 3   Name         418 non-null    object  
 4   Sex          418 non-null    category
 5   Age          332 non-null    float64 
 6   SibSp        418 non-null    int64   
 7   Parch        418 non-null    int64   
 8   Ticket       418 non-null    object  
 9   Fare         417 non-null    float64 
 10  Cabin        418 non-null    object  
 11  Embarked     418 non-null    category
dtypes: category(4), float64(2), int64(2), object(4)
memory usage: 28.4+ KB


**B.** Are there any missing values? If so,

- Study the impact of missing value removal on the quantitative variables.
- Study the impact of missing value removal on the qualitative variables.
- Conclude the dynamic of the missing values and handle them.
- Remove redundant observations if there is any.

In [15]:
df.isna().sum()

Survived     0
Pclass       0
Name         0
Sex          0
Age         86
SibSp        0
Parch        0
Ticket       0
Fare         1
Embarked     0
dtype: int64

In [ ]:
# Since Cabin is the most missing, even it's important, we still remove it
df = df.drop(columns=['PassengerId', 'Cabin'])

In [ ]:
from sklearn.impute import SimpleImputer

ip = SimpleImputer(strategy='mean')
df['Fare'] = sip.fit_transform(df[['Fare']])

In [13]:
object_columns = df.select_dtypes(include='object').columns.tolist()
print('Object columns: ', object_columns)
print()
number_columns = df.select_dtypes(include=['number', 'category']).columns.tolist()
print("Number columns: ", number_columns)
# print()
# category_columns = df.select_dtypes(includ)

Object columns:  ['Name', 'Ticket']

Number columns:  ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']


In [5]:
# To do

## 2. $\chi^2$-test and CA

The chi-square test is a statistical method used to determine if there is a significant association between two categorical variables. It tests the following hypotheses:
$$\begin{cases}
H_0:\text{ There is no association between the two variables (they are independent).}\\
H_1:\text{ There is an association between the two variables (they are not independent).}
\end{cases}$$
Under null hypothesis $H_0$, $\chi^2$-statistic defined by $\chi^2=\sum_{i,j}\frac{(O_{ij}-E_{ij})^2}{E_{ij}}\sim\chi^2((r-1)(c-1))$ where

- $r,c$: the number of categories of the 1st and 2nd variable respectively.
- $O_{ij}$: the observed frequency of $i$-th and $j$-th category of the 1st and the 2nd variable.
- $E_{ij}$: the expected/theoretical frequency of $i$-th and $j$-th category of the 1st and the 2nd variable.

**A. $\chi^2$-test for Pclass vs Survived.** 

- Visualize the relationship between the two variables.
- Compute the $\chi^2$ statistics of the pair `Pclass` and `Survived` variable.
- Deduce the p-value of $\chi^2$-test of the two variables.
- Can we reject the null hypothesis $H_0$ of the two variables being independent at $95\%$ confidence level?
- Recall the assumptions of $\chi^2$-test. Is the result above reliable?

In [6]:
import numpy as np
from scipy.stats import chi2_contingency
# To do

**B. Pclass vs Embarked:**

- Perform $\chi^2$-test on this pair of variables.
- Perform CA on this pair of variables.
- Create `symmetric biplot` of the resulting CA.
- Interpret the result.

In [7]:
# To do

## 3. Eye and Hair color

Study the connection between eye and hair colors from the `Eye & Hair Color` dataset available in kaggle as [Hair Eye Color](https://www.kaggle.com/datasets/jasleensondhi/hair-eye-color).

In [8]:
# To do

## 4. Countries and languages

Reproduce results of the association between countries and primary language spoken within those countries conducted [here](https://pmc.ncbi.nlm.nih.gov/articles/PMC3718710/). The contingency table of country of residence and primary language spoken is given below:
	
| Country\ Language  | English | French | Spanish | German | Italian | Total |
|:-------------------|:-------:|:------:|:-------:|:------:|:-------:|:-----:|
| Canada | 688 | 280 | 10 | 11 | 11 | 1000 |
| USA | 730 | 31 | 190 | 8 | 41 | 1000 |
| England | 798 | 74 | 38 | 31 | 59 | 1000 |
| Italy | 17 | 13 | 11 | 15 | 944 | 1000 |
| Switzerland | 15 | 222 | 20 | 648 | 95 | 1000 |
| Total | 2248 | 620 | 269 | 713 | 1150 | 5000 |

In [15]:
# To do

# Further Readings

- [Correspondence Analysis, Hervé Abdi & Michel Béra](https://cedric.cnam.fr/fichiers/art_3066.pdf)
- [Correspondence analysis is a useful tool to uncover the relationships among categorical variables](https://pmc.ncbi.nlm.nih.gov/articles/PMC3718710/)
- [The Use of Correspondence Analysis in the Exploration of Health Survey Data](www.fbbva.es/wp-content/uploads/2017/05/dat/DT_2002_05.pdf)
- [Correspondence Analysis](https://link.springer.com/referenceworkentry/10.1007/978-1-4939-7131-2_140)